# 1. Introdução

### Motivação Hipotética

Parabéns! Você acabou de ser contratado para liderar os esforços de data science na DataSciencester, a rede social para cientistas de dados.

Apesar de ser para os cientistas de dados, a DataSciencester nunca investiu em construir sua própria atividade de data science (na verdade, a DataSciencester nunca investiu em construir seu próprio produto). Esse será seu trabalho!

E, devido à DataSciencester possuir uma forte mentalidade de “não-foi-inventado-aqui”, nós construiremos nossas próprias ferramentas do zero. 

Bem-vindo a bordo e boa sorte! ‘Você pode usar jeans às sextas e o toalete é no final do corredor à direita.’

### Encontrando Conectores-Chave

É seu primeiro dia de trabalho na DataSciencester e o vice-presidente de Rede (networking) está cheio de perguntas sobre seus usuários.

Particularmente, ele quer que você identifique quem são os “conectores-chave” entre os cientistas de dados. Para isso, ele lhe dá uma parte de toda a rede da DataSciencester. (Capítulo 9 é voltado para a obtenção de dados)

Veja os dados abaixo:

In [1]:
# Usuários da Rede DataSciencester

users = [
    { "id": 0, "name": "Hero" },
    { "id": 1, "name": "Dunn" },
    { "id": 2, "name": "Sue" },
    { "id": 3, "name": "Chi" },
    { "id": 4, "name": "Thor" },
    { "id": 5, "name": "Clive" },
    { "id": 6, "name": "Hicks" },
    { "id": 7, "name": "Devin" },
    { "id": 8, "name": "Kate" },
    { "id": 9, "name": "Klein" }
]

Ele também fornece dados “amigáveis”, representados por uma lista de pares de IDs:

In [2]:
# Relacionamentos entre os usuários 
friendships = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
(4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]

Por exemplo, a tupla (0,1) indica que o cientista de dados com a id 0 (Hero) e o cientista de dados com a id 1 (Dunn) são amigos. 

Como os dados são representados por dicts, é fácil adicionar dados extras.

Por exemplo, talvez nós queiramos adicionar uma lista de amigos para cada
usuário. Primeiro nós configuramos a propriedade friends de cada usuário em uma
lista vazia:

In [3]:
# configurando um atributo "friends" no dict de users adicionando listas vazias para cada um
for user in users:
    user["friends"] = []

E povoamos essa lista com os dados de relacionamento:

In [4]:
# Povoando a lista de relacionamentos
for i, j in friendships: # Percorre a lista de tuplas de relação, pegando os ids dos usuários.
    users[i]["friends"].append(users[j]) # Atribui j aos amigos de i
    users[j]["friends"].append(users[i]) # Atribui i aos amigos de j

Em seguida, podemos perguntar: “qual é o número médio de conexões?”

Primeiro encontramos o número total de conexões e em seguida, dividimos pelo número de usuários: 

In [5]:
from __future__ import division

# Função para encontrar o número de amigos de cada usuário.
def number_of_friends(user):
    return len(user['friends']) # Retorna o tamanho da lista de amigos de um usuário

# Descobrindo o Total de Conexões:
    # Somatório do número de amigos de cada usuário da rede
total_connections = sum(number_of_friends(user) for user in users)

# Encontrando o número médio de conexões
num_users = len(users) # Número de usuários da rede
avg_connections = total_connections/num_users # Média de Conexões por usuário

Encontrando as pessoas mais conectadas ou seja, com o maior número de amigos (Para poucos dados):

In [6]:
# Ordenando em ordem decrescente pela quantidade de amigos
num_friends_by_id = [(user["id"], number_of_friends(user)) for user in users]
num_friends_by_id.sort(key=lambda x: x[1], reverse=True) # Diferente do livro, mesmo resultado

### Cientistas de Dados que talvez você conheça

Enquanto você está preenchendo os papéis de admissão, a vice-presidente da Fraternidade chega a sua mesa. Ela quer estimular mais conexões entre os seus membros, e pede que você desenvolva sugestões  de “Cientistas de Dados Que Você Talvez Conheça”.

Seu primeiro instinto é sugerir um usuário que possa conhecer amigos de amigos. São fáceis de computar: para cada amigo de um usuário, itera sobre os amigos daquela pessoa, e coleta todos os resultados:

In [7]:
# Forma ruim de encontrar amigos em comum
    # Pega os amigos de cada amigo do usuário
def friends_of_friend_ids_bad(user):
    return [foaf["id"] 
            for friend in user["friends"] 
            for foaf in friend["friends"]]

friends_of_friend_ids_bad(users[0]) # Amigos em Comum de Hero

[0, 2, 3, 0, 1, 3]

Entretanto, o código acima não é uma boa opção pois duplica os ids uma vez que pega todos os amigos dos seus amigos, incluindo ele mesmo. Veja os exemplos abaixo:

In [8]:
print([friend["id"] for friend in users[0]["friends"]]) 
print([friend["id"] for friend in users[1]["friends"]]) 
print([friend["id"] for friend in users[2]["friends"]]) 

[1, 2]
[0, 2, 3]
[0, 1, 3]


Saber que as pessoas são amigas-de-amigas de diversas maneiras parece uma informação interessante, então talvez nós devêssemos produzir uma contagem de amigos em comum. As funções abaixo elimina duplicatas e os que já são amigos. Além disso, faz uma contagem de amigos em comum.

In [9]:
from collections import Counter # Função de contagem não carregada por padrão

# Verifica se dois usuários não são os mesmos, retorna True ou False
def not_the_same(user, other_user):
    # Dois usuários não são os mesmos se possuem ids diferentes
    return user["id"] != other_user["id"]

# Verifica se dois usuários não são amigos
def not_friends(user, other_user):
    return all(not_the_same(friend, other_user) 
               for friend in user["friends"])

# Função para encontrar amigos em comum
def friends_of_friend_ids(user):
    return Counter(foaf["id"]
                  for friend in user["friends"] 
                  for foaf in friend["friends"] 
                  if not_the_same(user, foaf)   
                   and not_friends(user, foaf))

print(friends_of_friend_ids(users[3]))

Counter({0: 2, 5: 1})


Como um cientista de dados, você sabe que você pode gostar de encontrar usuários com interesses similares. Depois de perguntar por aí, você consegue pôr as mãos nesse dado, como uma lista de pares (user_id, interest) :

In [10]:
interests = [
(0, "Hadoop"), (0, "Big Data"), (0, "HBase"), (0, "Java"), (0, "Spark"), (0, "Storm"), 
(0, "Cassandra"),
(1, "NoSQL"), (1, "MongoDB"), (1, "Cassandra"), (1, "HBase"), (1, "Postgres"), 
(2, "Python"), (2, "scikit-learn"), (2, "scipy"), (2, "numpy"), (2, "statsmodels"), (2, "pandas"), 
(3, "R"), (3, "Python"), (3, "statistics"), (3, "regression"), (3, "probability"),
(4, "machine learning"), (4, "regression"), (4, "decision trees"), (4, "libsvm"), 
(5, "Python"), (5, "R"), (5, "Java"), (5, "C++"), (5, "Haskell"), (5, "programming languages"), 
(6, "statistics"), (6, "probability"), (6, "mathematics"), (6, "theory"),
(7, "machine learning"), (7, "scikit-learn"), (7, "Mahout"), (7, "neural networks"), 
(8, "neural networks"), (8, "deep learning"),(8, "Big Data"), (8, "artificial intelligence"), 
(9, "Hadoop"), (9, "Java"), (9, "MapReduce"), (9, "Big Data")
]

A função abaixo encontra usuários com o mesmo interesse:

In [11]:
# Função que retorna os ids de pessoas com interesse em determinado tema
def data_scientists_who_like(target_interest):
    return [user_id
            for user_id, user_interest in interests
            if user_interest == target_interest]

print(data_scientists_who_like("Hadoop"))
print(data_scientists_who_like("Cassandra"))

[0, 9]
[0, 1]


A função resolve, mas para grandes quantidades de dados, se tornaria inviável. Para esse caso, é melhor utilizar a abordagem de indexação apresentada abaixo: 

In [12]:
from collections import defaultdict

# Dicionário de listas de usuários com um mesmo interesse
# As chaves são interesses e os valores são os ids dos usuários
user_ids_by_interest = defaultdict(list)

for user_id, interest in interests:
    user_ids_by_interest[interest].append(user_id)

# Dicionário de listas de interesses de um usuário
# As chaves são ids dos usuários e os valores são os interesses 
interests_by_user_id = defaultdict(list)

for user_id, interest in interests:
    interests_by_user_id[user_id].append(interest)

In [13]:
user_ids_by_interest

defaultdict(list,
            {'Hadoop': [0, 9],
             'Big Data': [0, 8, 9],
             'HBase': [0, 1],
             'Java': [0, 5, 9],
             'Spark': [0],
             'Storm': [0],
             'Cassandra': [0, 1],
             'NoSQL': [1],
             'MongoDB': [1],
             'Postgres': [1],
             'Python': [2, 3, 5],
             'scikit-learn': [2, 7],
             'scipy': [2],
             'numpy': [2],
             'statsmodels': [2],
             'pandas': [2],
             'R': [3, 5],
             'statistics': [3, 6],
             'regression': [3, 4],
             'probability': [3, 6],
             'machine learning': [4, 7],
             'decision trees': [4],
             'libsvm': [4],
             'C++': [5],
             'Haskell': [5],
             'programming languages': [5],
             'mathematics': [6],
             'theory': [6],
             'Mahout': [7],
             'neural networks': [7, 8],
             'deep learning': 

In [14]:
interests_by_user_id

defaultdict(list,
            {0: ['Hadoop',
              'Big Data',
              'HBase',
              'Java',
              'Spark',
              'Storm',
              'Cassandra'],
             1: ['NoSQL', 'MongoDB', 'Cassandra', 'HBase', 'Postgres'],
             2: ['Python',
              'scikit-learn',
              'scipy',
              'numpy',
              'statsmodels',
              'pandas'],
             3: ['R', 'Python', 'statistics', 'regression', 'probability'],
             4: ['machine learning', 'regression', 'decision trees', 'libsvm'],
             5: ['Python',
              'R',
              'Java',
              'C++',
              'Haskell',
              'programming languages'],
             6: ['statistics', 'probability', 'mathematics', 'theory'],
             7: ['machine learning',
              'scikit-learn',
              'Mahout',
              'neural networks'],
             8: ['neural networks',
              'deep learning',
       

Agora fica fácil descobrir quem possui os maiores interesses em comum com um certo usuário:
* Itera sobre os interesses do usuário.
* Para cada interesse, itera sobre os outros usuários com aquele interesse.
* Mantém a contagem de quantas vezes vemos cada outro usuário.

In [15]:
# Retorna os usuários com interesses em comum com um determinado usuário
def most_common_interests_with(user):
    return Counter(interested_user_id
                    for interest in interests_by_user_id[user["id"]]
                    for interested_user_id in user_ids_by_interest[interest]
                    if interested_user_id != user["id"])

most_common_interests_with(users[0])

Counter({9: 3, 8: 1, 1: 2, 5: 1})

### Salários e Experiência

Na hora em que você está saindo para o almoço, o vice-presidente de Relações Públicas pergunta se você pode fornecer alguns fatos curiosos sobre quanto os cientistas de dados recebem. Dados de salário é, de fato, um tópico sensível, mas ele consegue fornecer um conjunto de dados anônimos contendo o salary (salário) de cada usuário (em dólares) e tenure (experiência) como um cientista de dados (em anos):

In [16]:
salaries_and_tenures = [
    (83000, 8.7), (88000, 8.1),
    (48000, 0.7), (76000, 6),
    (69000, 6.5), (76000, 7.5),
    (60000, 2.5), (83000, 10),
    (48000, 1.9), (63000, 4.2)
]
salaries_and_tenures.sort(key=lambda x: x[1])

A primeira ideia é analisar a média salarial para cada ano:

In [17]:
# Média Salarial para cada ano

# As chaves são anos e os valores são as listas dos salários
salary_by_tenure = defaultdict(list)

for salary, tenure in salaries_and_tenures:
    salary_by_tenure[tenure].append(salary)
    
# as chaves são os anos, cada valor é a média salarial para aquele ano
average_salary_by_tenure = {
    tenure : sum(salaries) / len(salaries)
    for tenure, salaries in salary_by_tenure.items()
}

Não é muito útil, já que nenhum dos usuários possui o mesmo caso, o que significa que estamos reportando apenas os salários individuais dos usuários:

In [18]:
average_salary_by_tenure

{0.7: 48000.0,
 1.9: 48000.0,
 2.5: 60000.0,
 4.2: 63000.0,
 6: 76000.0,
 6.5: 69000.0,
 7.5: 76000.0,
 8.1: 88000.0,
 8.7: 83000.0,
 10: 83000.0}

Uma ideia melhor seria agrupar esses dados, como realizado abaixo:

In [19]:
def tenure_bucket(tenure): # O agrupamento é feito segundo essas condições
    if tenure < 2:
        return "less than two"
    elif tenure < 5:
        return "between two and five"
    else:
        return "more than five"
    
# as chaves são agrupamentos dos casos, os valores são as listas
# dos salários para aquele agrupamento
salary_by_tenure_bucket = defaultdict(list)

for salary, tenure in salaries_and_tenures:
    bucket = tenure_bucket(tenure)
    salary_by_tenure_bucket[bucket].append(salary)
    
# as chaves são agrupamentos dos casos, os valores são
# a média salarial para aquele agrupamento
average_salary_by_bucket = {
    tenure_bucket : sum(salaries) / len(salaries)
    for tenure_bucket, salaries in salary_by_tenure_bucket.items()
}

E obtemos os seguintes grupos:

In [20]:
average_salary_by_bucket

{'less than two': 48000.0,
 'between two and five': 61500.0,
 'more than five': 79166.66666666667}

### Contas Pagas

Ao voltar para a sua mesa, a vice-presidente da Receita está esperando por você. Ela quer entender melhor quais são os usuários que pagam por contas e quais que não pagam (ela sabe seus nomes, mas essa informação não é essencial).

Percebe-se que parece haver uma correspondência entre os anos de experiência e as contas pagas:

In [21]:
dados = [
    (0.7, "paid"),
    (1.9, "unpaid"),
    (2.5, "paid"),
    (4.2, "unpaid"),
    (6, "unpaid"),
    (6.5, "unpaid"),
    (7.5, "unpaid"),
    (8.1, "unpaid"),
    (8.7, "paid"),
    (10, "paid")
]
dados.sort(key=lambda x: x[0])

In [22]:
dados

[(0.7, 'paid'),
 (1.9, 'unpaid'),
 (2.5, 'paid'),
 (4.2, 'unpaid'),
 (6, 'unpaid'),
 (6.5, 'unpaid'),
 (7.5, 'unpaid'),
 (8.1, 'unpaid'),
 (8.7, 'paid'),
 (10, 'paid')]

Os usuários com poucos e muitos anos de experiência tendem a pagar; os usuários com uma quantidade mediana de experiência não.

Logo, se você quisesse criar um modelo — apesar de não haver dados o suficiente para servir de base para um — você talvez tentasse prever “paid” para os usuários com poucos e muitos anos de experiência, e “unpaid” para os usuários com quantidade mediana de experiência:

In [23]:
def predict_paid_or_unpaid(years_experience):
    if years_experience < 3.0:
        return "paid"
    elif years_experience < 8.5:
        return "unpaid"
    else:
        return "paid"
    
predict_paid_or_unpaid(5)

'unpaid'

Quando seu dia está terminando, a vice-presidente da Estratégia de Conteúdo pede dados sobre em quais tópicos os usuários estão mais interessados, para que ela possa planejar o calendário do seu blog de acordo.

Uma simples forma (e também fascinante) de encontrar os interesses mais populares é fazer uma simples contagem de palavras:
1. Coloque cada um em letras minúsculas (já que usuários diferentes podem ou não escrever seus interesses em letras maiúsculas).
2. Divida em palavras.
3. Conte os resultados.

Utilizando os dados de interesse usados anteriormente:

In [24]:
words_and_counts = Counter(word 
                           for user, interest in interests 
                           for word in interest.lower().split())

O que torna possível a listagem de palavras que ocorrem com maior frequência:

In [25]:
for word, count in words_and_counts.most_common():
    if count > 1:
        print (word, count)

big 3
data 3
java 3
python 3
learning 3
hadoop 2
hbase 2
cassandra 2
scikit-learn 2
r 2
statistics 2
regression 2
probability 2
machine 2
neural 2
networks 2


### Em diante

Foi um primeiro dia bem proveitoso! Exausto, você sai do prédio antes que alguém peça algo mais. Tenha uma boa noite de sono, porque amanhã será dia de treinamento para novos funcionários. Sim, você trabalhou um dia inteiro antes do treinamento. Culpa do RH.